In [ ]:
!pip install ultralytics deep-sort-realtime opencv-python-headless ffmpeg-python

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from IPython.display import HTML, display
from base64 import b64encode
import os

In [ ]:
# Load YOLOv8 lightweight model (nano version)
from ultralytics import YOLO
yolo_model = YOLO("yolov8n.pt")

# Ganti dengan 'yolov8s.pt' untuk akurasi lebih tinggi

# Inisialisasi pelacak DeepSORT
from deep_sort_realtime.deepsort_tracker import DeepSort
object_tracker = DeepSort(
    max_age=30,
    n_init=3,
    max_iou_distance=0.7,
    embedder="mobilenet",
    half=True,
    bgr=True
)


In [ ]:
print("📤 Please upload your video file...")
data_video = files.upload()
video_path = list(data_video.keys())[0]
print(f"✅ Uploaded video: {video_path}")

In [ ]:
# Buka file video
video_capture = cv2.VideoCapture(video_path)
if not video_capture.isOpened():
    raise ValueError("❌ Tidak dapat membuka file video!")

# Ambil properti video
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_fps = video_capture.get(cv2.CAP_PROP_FPS)
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# Tampilkan informasi video
print(f"📐 Resolusi video: {video_width}x{video_height}")
print(f"🎬 {video_fps} FPS, total {total_frames} frame")
print(f"⏳ Durasi estimasi: {total_frames / video_fps:.2f} detik")

# Siapkan output video
save_path = 'output_counting.mp4'
codec = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(save_path, codec, video_fps, (video_width, video_height))


In [ ]:
# Buka video
cap = cv2.VideoCapture(video_path)  # Tambahan penting untuk menghindari NameError

counted_ids = set()
total_count = 0
frame_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_num += 1
    if frame_num % 10 == 0:
        print(f"⏳ Processing frame {frame_num}/{total_frames}", end='\r')

    # Ubah BGR ke RGB untuk YOLO
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Deteksi objek manusia (class 0 = person)
    results = yolo_model(frame_rgb, classes=[0], verbose=False)[0]
    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0].item()
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # Update pelacakan objek
    tracks = object_tracker.update_tracks(detections, frame=frame_rgb)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())

        # Gambar kotak pelacakan
        cv2.rectangle(frame, (l, t), (r, b), (0, 200, 255), 2)
        cv2.putText(frame, f'ID:{track_id}', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Hitung hanya ID yang belum tercatat
        if track_id not in counted_ids:
            counted_ids.add(track_id)
            total_count += 1

    # Tampilkan total
    cv2.putText(frame, f'Total: {total_count}', (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # Simpan frame hasil
    video_writer.write(frame)

cap.release()
video_writer.release()
print(f"\n✅ Processing complete! Total people counted: {total_count}")

In [ ]:
# Check output file
output_size = os.path.getsize(save_path)/1024/1024
print(f"📁 Output file size: {output_size:.2f} MB")

if output_size < 0.1:  # If file is too small
    print("⚠️ Warning: Output file seems too small. There might be an issue.")
else:
    print("✔️ Output file looks good")

In [ ]:
def show_video(video_path):
    # Convert to compatible format
    !ffmpeg -y -i {video_path} -vcodec libx264 output_final.mp4 -hide_banner -loglevel error

    # Display in notebook
    mp4 = open("output_final.mp4", "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f'''
    <div style="margin: 10px 0;">
      <video width="800" controls>
        <source src="{data_url}" type="video/mp4">
      </video>
      <p>Total people counted: {total_count}</p>
    </div>
    '''))

show_video(save_path)

In [ ]:
from google.colab import files
files.download(save_path)